In [75]:
#!pip install sqlite pandas numpy

In [2]:
import pandas as pd
import numpy as np
import os
import sqlite3

### Вспомогательные функции для работы с базой данных SQLite

In [ ]:
def create_batabase(basename):
    try:
        sqlite_connection = sqlite3.connect(basename+'.db')
        cursor = sqlite_connection.cursor()
        print("База данных создана и успешно подключена к SQLite")
        sqlite_select_query = "select sqlite_version();"
        cursor.execute(sqlite_select_query)
        record = cursor.fetchall()
        print("Версия базы данных SQLite: ", record)
        cursor.close()
    except sqlite3.Error as error:
        print("Ошибка при подключении к sqlite", error)
    finally:
        if (sqlite_connection):
            sqlite_connection.close()
            print("Соединение с SQLite закрыто")
                       
def execute(sql,basename):
    sqlite_connection = sqlite3.connect(basename+'.db', timeout=10)
    cursor = sqlite_connection.cursor()
    cursor.execute(sql)
    sqlite_connection.commit()
    cursor.close()

def retexec(sql,basename):
    sqlite_connection = sqlite3.connect(basename+'.db', timeout=10)
    cursor = sqlite_connection.cursor()
    cursor.execute(sql)
    cur = cursor.fetchall()
    r = pd.DataFrame(cur)
    if cursor is not None and isinstance (cursor, sqlite3.Cursor):
        colnames = [row[0] for row in cursor.description]
        if len(r.columns) == len(colnames):
            r.columns = colnames
    sqlite_connection.commit()
    cursor.close()
    return r

def fill_table_many(table_name, dbs, datecolumns,basename):
    start = datetime.now()
    data = dbs.copy()
    for c in data.columns:
        data[c] = data[c].apply(str)
    ve = [x.upper() for x in data.columns]
    vc = '", "'.join(ve)
    vc = '("'+vc+'")'
    vd = '", :"'.join(ve)
    vd = '(:"'+vd+'")'    
    ve = ['?' for x in data.columns]
    vc = ', '.join(ve)
    vc = '('+vc+')'
    sql = ("""insert into """+table_name+""" values """+vc)
    result = []
    for j in tqdm(range(len(data))):
        res = [data.loc[j, c] for c in data.columns]
        res = tuple(i for i in res)
        result.append(res)
    try:
        connection = sqlite3.connect(basename+'.db', timeout=10)
        cur = connection.cursor()
        cur.executemany(sql,  result)
        connection.commit()
        cur.close()
    except:
        print(cur.rowcount)
    finish = datetime.now()
    t = '{}'.format(finish-start)
    print(t)

### Путь рабочего каталога

In [3]:
os.getcwd()

'/Users/lmv'

### Создание таблицы с данными

In [5]:
STAFF = pd.DataFrame({
    'ID':[1,2,3,4, 5], 
    'ID_LEAD': [2,2,3,4,2],
    'ID_DEP': [1,1,3,4,1],
    'NAME': ['ИВАН', 'МАКСИМ', 'АЛЕКСАНДРА', 'МАРИЯ', 'АЛЕНА'],
    'SALARY': [94000.00, 138000.33, 192003.22, 57800, 200000],
})
print(STAFF)

   ID  ID_LEAD  ID_DEP        NAME     SALARY
0   1        2       1        ИВАН   94000.00
1   2        2       1      МАКСИМ  138000.33
2   3        3       3  АЛЕКСАНДРА  192003.22
3   4        4       4       МАРИЯ   57800.00
4   5        2       1       АЛЕНА  200000.00


### Сохранение и чтение CSV

In [7]:
STAFF.to_csv('my_dataset.csv', index=False, sep=';', encoding='cp1251')

In [69]:
df = pd.read_csv('my_dataset.csv', sep=';', encoding='cp1251')
print(df)

   ID  ID_LEAD  ID_DEP        NAME     SALARY
0   1        2       1        ИВАН   94000.00
1   2        2       1      МАКСИМ  138000.33
2   3        3       3  АЛЕКСАНДРА  192003.22
3   4        4       4       МАРИЯ   57800.00
4   5        2       1       АЛЕНА  200000.00


### Сохранение и чтение PKL

In [19]:
df.to_pickle('my_dataset.pkl')

df2 = pd.read_pickle('my_dataset.pkl')

df.equals(df2)

True

### Сохранение и чтение XLSX

In [16]:
df.to_excel("output.xlsx", sheet_name='Sheet_name_1', index=False)

In [17]:
dbs = pd.read_excel("output.xlsx", sheet_name='Sheet_name_1')
dbs.head()

,ID,ID_LEAD,ID_DEP,NAME,SALARY
0,1,2,1,ИВАН,94000.00
1,2,2,1,МАКСИМ,138000.33
2,3,3,3,АЛЕКСАНДРА,192003.22
3,4,4,4,МАРИЯ,57800.00
4,5,2,1,АЛЕНА,200000.00


In [70]:
df.equals(dbs)

True

### Сохранение и чтение XLSX, несколько листов

In [77]:
df2 = df.copy()
with pd.ExcelWriter('output.xlsx') as writer:  
    df.to_excel(writer, sheet_name='Sheet_name_1', index=False)
    df2.to_excel(writer, sheet_name='Sheet_name_2', index=False)

In [78]:
db1 = pd.read_excel("output.xlsx", sheet_name='Sheet_name_1')
db2 = pd.read_excel("output.xlsx", sheet_name='Sheet_name_2')

### Сохранение в базу данных SQLite

In [21]:
basename = 'sqlite_python_new'
create_batabase(basename)

База данных создана и успешно подключена к SQLite
Версия базы данных SQLite:  [('3.33.0',)]
Соединение с SQLite закрыто


In [67]:
categorical_columns = [c for c in df.columns if df[c].dtype.name == 'object']
numerical_columns   = [c for c in df.columns if df[c].dtype.name != 'object']
numerical_int_columns   = [c for c in numerical_columns if str(np.dtype(df[c])).count('int') >  0 ]
numerical_flt_columns   = [c for c in numerical_columns if str(np.dtype(df[c])).count('int') == 0 ]
print('Категориальные признаки', categorical_columns)
print('Числовые целые признаки', numerical_int_columns)
print('Числовые дробные признаки', numerical_flt_columns)

Категориальные признаки ['NAME']
Числовые целые признаки ['ID', 'ID_LEAD', 'ID_DEP']
Числовые дробные признаки ['SALARY']


In [52]:
types = []
for c in df.columns:
    if c in categorical_columns: types.append('TEXT')
    if c in numerical_int_columns: types.append('INTEGER')
    if c in numerical_flt_columns: types.append('Decimal')
print(types)

['INTEGER', 'INTEGER', 'INTEGER', 'TEXT', 'Decimal']


### Создание таблицы

In [56]:
fields = list(df.columns)
table_name = 'STAFF'
s = []
for i in range(len(fields)):
    s.append(' '.join([fields[i], types[i]]))
s = ', '.join(s)

sql = """CREATE TABLE IF NOT EXISTS """+table_name+""" ( """+s+""");"""
execute(sql, basename)

### Чтение из базы данных

In [57]:
dfs = retexec("""select * from """+table_name, basename)
print(dfs)

In [58]:
dfs

""


### Запись данных в таблицу

In [61]:
datecolumns=[]
fill_table_many(table_name, df, datecolumns, basename)

100%|██████████| 5/5 [00:00<00:00, 1679.47it/s]

0:00:00.030040


In [62]:
dfs = retexec("""select * from """+table_name, basename)
print(dfs)

   ID  ID_LEAD  ID_DEP        NAME     SALARY
0   1        2       1        ИВАН   94000.00
1   2        2       1      МАКСИМ  138000.33
2   3        3       3  АЛЕКСАНДРА  192003.22
3   4        4       4       МАРИЯ   57800.00
4   5        2       1       АЛЕНА  200000.00


### Запись и чтение массивов Numpy из файла

In [63]:
#Массивы numpy можно сохранять в файлы с расширением '.npy' и читать из таких файлов.
a = np.array([[7, 4, 5],
              [8, 3, 2],
              [6, 10, 12]])

np.save("a.npy", a)

#Для чтения из файла используется функция `np.load`:

a = np.load("a.npy")

a

array([[ 7,  4,  5],
       [ 8,  3,  2],
       [ 6, 10, 12]])

### Чтение из URL

In [65]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data'
data = pd.read_csv(url, header=None, na_values='?')

In [66]:
print(data)

    0      1       2  3  4   5   6     7  8  9   10 11 12     13   14 15
0    b  30.83   0.000  u  g   w   v  1.25  t  t   1  f  g  202.0    0  +
1    a  58.67   4.460  u  g   q   h  3.04  t  t   6  f  g   43.0  560  +
2    a  24.50   0.500  u  g   q   h  1.50  t  f   0  f  g  280.0  824  +
3    b  27.83   1.540  u  g   w   v  3.75  t  t   5  t  g  100.0    3  +
4    b  20.17   5.625  u  g   w   v  1.71  t  f   0  f  s  120.0    0  +
..  ..    ...     ... .. ..  ..  ..   ... .. ..  .. .. ..    ...  ... ..
685  b  21.08  10.085  y  p   e   h  1.25  f  f   0  f  g  260.0    0  -
686  a  22.67   0.750  u  g   c   v  2.00  f  t   2  t  g  200.0  394  -
687  a  25.25  13.500  y  p  ff  ff  2.00  f  t   1  t  g  200.0    1  -
688  b  17.92   0.205  u  g  aa   v  0.04  f  f   0  f  g  280.0  750  -
689  b  35.00   3.375  u  g   c   h  8.29  f  f   0  t  g    0.0    0  -

[690 rows x 16 columns]
